In [3]:
#lee logs de ingestas de tableros de producción

In [4]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
import parametros

In [5]:
#FECHA ACTUAL 

In [6]:
now = datetime.now()
ahora = str(now.strftime("%Y-%m-%dT%H:%M:%S"))

In [7]:
#conecta elasticsearch

In [8]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

In [9]:
#Leyendo archivo log trafico

In [10]:
 try:
    #logread = pd.read_csv('../archivos/sonicwall-curado.log', sep='\t', encoding = 'utf-8')
    header_list = ["column1"]
    logread = pd.read_csv('../archivos/ETL-Trafico.log', header=None,names=header_list, encoding = 'utf-8')
    #logread = pd.read_csv('../logs/ETL-Trafico.log', header=None,names=header_list, encoding = 'utf-8')
    logread.fillna('', inplace=True)
   
 except:
    print("fecha:",ahora,"- Error leyendo archivo trafico.log")
    exit()

In [11]:
logread1 = logread["column1"].str.split(expand=True)

In [12]:

logread1 = logread["column1"].str.split(expand=True)
logread1.columns = ['fecha ', 'last_name', 'last_name2', 'last_name3', 'last_name4', 'last_name5', 'last_name6', 'last_name7',  'last_name8', 'last_name9', 'last_name10', 'last_name11', 'last_name12', 'last_name13']

logread = pd.concat([logread, logread1], axis=1)

In [13]:
logread["d_last_name"] = logread['last_name']+logread['last_name2']

In [14]:
logread["d_last_name2"] = logread['last_name11']+logread['last_name12']

In [15]:
logread.fillna('', inplace=True)

In [16]:
#logread_last_ = logread.tail(5)

In [17]:
#list(logread1.columns)

In [18]:
indice = 'alarma_trafico_4'

In [19]:
use_these_keys = ['last_name', 'last_name2', 'last_name9', 'last_name11', '@timestamp']

In [20]:
use_these_keys = ['d_last_name', 'd_last_name2', 'last_name9', 'last_name13', '@timestamp' ]
def filterKeys(document):
   return {key: document[key] for key in use_these_keys }
timestamp = datetime.now()
logread['@timestamp'] = timestamp.isoformat()



def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": index,
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(logread))
print("fecha:",ahora,"- Insertados en monitor-trafico:", salida[0])




fecha: 2021-12-10T17:10:41 - Insertados en monitor-trafico: 74308
